Requisitos

In [242]:
!pip install matplotlib
!pip install pandas

In [243]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

Resultados de los torneos cargados directamente desde GitHub

In [244]:
nice_or_die_dom = "https://github.com/luiserh1/NGA/raw/main/log/dom1.csv"
grocery_dom = "https://github.com/luiserh1/NGA/raw/main/log/dom2.csv"
energy_dom = "https://github.com/luiserh1/NGA/raw/main/log/dom3.csv"

In [245]:
dom1_df = pd.read_csv(nice_or_die_dom, sep=';')
dom2_df = pd.read_csv(grocery_dom, sep=';')
#dom3_df = pd.read_csv(energy_dom)

In [246]:
print(dom1_df[:2])
print(dom2_df[:2])

  Run time (s)  Round  Exception  ... User Util. 2       Profile 1       Profile 2
0        0,986   1001        NaN  ...          0.0  NiceOrDie1.xml  NiceOrDie2.xml
1        0,674   1001        NaN  ...          0.0  NiceOrDie1.xml  NiceOrDie2.xml

[2 rows x 26 columns]
  Run time (s)  Round  ...               Profile 1                Profile 2
0        0,528   1001  ...  Grocery_domain_sam.xml  Grocery_domain_mary.xml
1        0,461    732  ...  Grocery_domain_sam.xml  Grocery_domain_mary.xml

[2 rows x 26 columns]


Extraenos la información que nos interesa

In [247]:
print(dom1_df.columns)

Index(['Run time (s)', 'Round', 'Exception', 'deadline', 'Agreement',
       'Discounted', '#agreeing', 'min.util.', 'max.util.', 'Dist. to Pareto',
       'Dist. to Nash', 'Social Welfare', 'Agent 1', 'Agent 2', 'Utility 1',
       'Utility 2', 'Disc. Util. 1', 'Disc. Util. 2', 'Perceived. Util. 1',
       'Perceived. Util. 2', 'User Bother 1', 'User Bother 2', 'User Util. 1',
       'User Util. 2', 'Profile 1', 'Profile 2'],
      dtype='object')


Definimos la rejilla y una serie de identificadores para los hiperparámtros
Las claves de la rejilla son tuplas de 4 valores

In [248]:
BIDDING_STRATEGY = 0
ACCEPTANCE_STRATEGY = 1
OPPONENT_MODEL = 2
OPPONENT_MODEL_STRATEGY = 3

OPPONENT = 0
AGREEMENT = 1
UTILITY = 2
RUN_TIME = 3
IS_CHALLENGE = 4
PROFILE = 5

In [249]:
def clean_agent_name(text):
    text = text[:text.index("@")]
    challenger = True
    try:
      text = text[text.index("anac201")+9:]
      text = text.replace(".anac2011.","")
      text = "\t\t".join(text.split(".boaframework."))
      text = text.replace("_Offering","")
      text = text.replace("-negotiator","")
      text = text.replace("acceptanceconditions","")
      text = text.replace("opponentmodel.","")
      text = text.replace("omstrategy.","")
      text = text.replace("_t_n","")
      text = text.replace("_n_t","")
    except ValueError:
      challenger = False
    return (text, challenger)

In [250]:
def get_grid(data_frame):
  grid = {}
  for i in range(len(data_frame)):
    agent1 = data_frame.at[i, "Agent 1"]
    agent1, challenger1 = clean_agent_name(agent1)
    agent2 = data_frame.at[i, "Agent 2"]
    agent2, challenger2 = clean_agent_name(agent2)
    utility1 = float(data_frame.at[i, "Utility 1"])
    utility2 = float(data_frame.at[i, "Utility 2"])
    profile1 = data_frame.at[i, "Profile 1"]
    profile2 = data_frame.at[i, "Profile 2"]
    run_time = float(data_frame.at[i, "Run time (s)"].replace(",", "."))
    agreement = data_frame.at[i, "Agreement"] == "Yes"
    if challenger1:
      hiperparameters = agent1.split("\t\t")
      key = tuple(hiperparameters)
      rows = grid.get(key, [])
      row = tuple([agent2, agreement, utility1, run_time, not challenger2,
                   profile1])
      rows.append(row)
      grid[key] = rows
    if challenger2:
      hiperparameters = agent2.split("\t\t")
      key = tuple(hiperparameters)
      rows = grid.get(key, [])
      row = tuple([agent1, agreement,utility2, run_time, not challenger1,
                   profile2])
      rows.append(row)
      grid[key] = rows
  return grid

In [255]:
grid1 = get_grid(dom1_df)

In [256]:
for key in grid1.keys():
  print(key)

('Nozomi', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')
('TheNegotiator', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')
('Nozomi', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')
('TheNegotiator', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')
('Nozomi', 'AC_IAMHaggler2011', 'UniformModel', 'OfferBestN')
('TheNegotiator', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')
('Nozomi', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')
('TheNegotiator', 'AC_IAMHaggler2011', 'UniformModel', 'OfferBestN')
('TheNegotiator', 'AC_IAMHaggler2011', 'BayesianModel_m', 'OfferBestN')
('Nozomi', 'AC_IAMHaggler2011', 'BayesianModel_m', 'OfferBestN')
('TheNegotiator', 'AC_HardHeaded', 'BayesianModel_m', 'OfferBestN')
('Nozomi', 'AC_HardHeaded', 'BayesianModel_m', 'OfferBestN')
('BRAMAgent', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')
('BRAMAgent', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')
('BRAMAgent', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')
('BRAMAgent', 'AC_IAMHaggler2011'

In [257]:
def get_profile_averages(grid):
  agent_profile_averages = dict()
  for agent in  grid.keys():
    rows = grid[agent]
    profiles = dict()
    for row in rows:
      profile = row[PROFILE]
      values = profiles.get(profile, [])
      values.append(row[UTILITY])
      profiles[profile] = values
    my_averages = dict()
    for profile in profiles.keys():
      average = sum(profiles[profile]) / len(profiles[profile])
      my_averages[profile] = average
    agent_profile_averages[agent] = my_averages
  return agent_profile_averages


def normalize_averages(prof_averages):
  profiles = None
  for agent in prof_averages:
    profiles = set(prof_averages[agent].keys())
  normalized_averages = dict()
  bounds_per_profile = dict()
  for profile in profiles:
    values = []
    for agent in prof_averages:
      values.append(prof_averages[agent][profile])
    bounds_per_profile[profile] = (min(values), max(values))
  for agent in prof_averages:
    normalized_averages[agent] = dict()
    for profile in profiles:
      normalized_averages[agent][profile] = \
        (prof_averages[agent][profile] - bounds_per_profile[profile][0]) / \
        (bounds_per_profile[profile][1] - bounds_per_profile[profile][0])
  return normalized_averages

In [258]:
prof_averages = get_profile_averages(grid1)
for agent in prof_averages:
  print("{}\t=>\t\t\t\t\t{}".format(agent, prof_averages[agent]))
print("\n==================================================\n")
norm_prof_averages = normalize_averages(prof_averages)
for agent in norm_prof_averages:
  print("{}\t=>\t\t\t\t\t{}".format(agent, norm_prof_averages[agent]))

('Nozomi', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.0, 'NiceOrDie2.xml': 0.045454545454545456}
('TheNegotiator', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.08834090909090908, 'NiceOrDie2.xml': 0.10872727272727274}
('Nozomi', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.0, 'NiceOrDie2.xml': 0.045454545454545456}
('TheNegotiator', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.09513636363636363, 'NiceOrDie2.xml': 0.10872727272727274}
('Nozomi', 'AC_IAMHaggler2011', 'UniformModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.0, 'NiceOrDie2.xml': 0.045454545454545456}
('TheNegotiator', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.08154545454545453, 'NiceOrDie2.xml': 0.10872727272727274}
('Nozomi', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')	=>					{'NiceOrDie1.xml': 0.0, 'NiceOrDie2.xml': 0.045454545454545456}
('TheNegotiator

In [259]:
def get_domain_averages(norm_prof_averages):
  profiles = None
  for agent in prof_averages:
    profiles = set(prof_averages[agent].keys())
  domain_averages = dict()
  for agent in norm_prof_averages.keys():
    values = np.zeros(len(profiles))
    for profile in profiles:
      values[0] = norm_prof_averages[agent][profile]
    mean = np.mean(values)
    interval = st.norm.interval(alpha=0.95, loc=mean, scale=st.sem(values))
    domain_averages[agent] = (interval[0], interval[1], mean)
  return domain_averages

In [260]:
domain_averages = get_domain_averages(norm_prof_averages)
for agent in domain_averages:
  print("{}\t=>\t\t\t\t\t{}".format(agent, domain_averages[agent]))

('Nozomi', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')	=>					(nan, nan, 0.0)
('TheNegotiator', 'AC_HardHeaded', 'UniformModel', 'OfferBestN')	=>					(-0.4456975642507393, 1.3742689928221679, 0.46428571428571425)
('Nozomi', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')	=>					(nan, nan, 0.0)
('TheNegotiator', 'AC_HardHeaded', 'OppositeModel', 'OfferBestN')	=>					(-0.479981992270027, 1.479981992270027, 0.5)
('Nozomi', 'AC_IAMHaggler2011', 'UniformModel', 'OfferBestN')	=>					(nan, nan, 0.0)
('TheNegotiator', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')	=>					(-0.41141313623145165, 1.2685559933743087, 0.42857142857142855)
('Nozomi', 'AC_IAMHaggler2011', 'OppositeModel', 'OfferBestN')	=>					(nan, nan, 0.0)
('TheNegotiator', 'AC_IAMHaggler2011', 'UniformModel', 'OfferBestN')	=>					(-0.4456975642507393, 1.3742689928221679, 0.46428571428571425)
('TheNegotiator', 'AC_IAMHaggler2011', 'BayesianModel_m', 'OfferBestN')	=>					(-0.41141313623145165, 1.2685559933743087, 0.428571428

/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [ ]:
grid1 = get_grid(dom1_df)
grid2 = get_grid(dom2_df)
grid3 = get_grid(dom3_df)